In [1]:
import numpy as np
from scipy.optimize import linprog
import networkx

In [2]:
A = np.array([[-1, -1, -1], [-1,2, 0], [0, 0, -1], [-1, 0, 0], [0, -1, 0]])
b = np.array([-1000, 0, -340, 0, 0])
c = np.array([10,15,25]) 

res = linprog(c, A_ub=A, b_ub=b,bounds=(0, None))

print('Optimal value:', res.fun, '\nX:', res.x)

('Optimal value:', 15100.0, '\nX:', array([660.,   0., 340.]))


In [3]:
G1 = {1:set({}), 2:set({}), 3:set({})}
G2 = {1:{2}, 2:{1}, 3:{}}
G3 = {1:{2}, 2:{1,3}, 3:{2}}
G4 = {1:{2,3}, 2:{1,3}, 3:{1,2}}

Define function to get subsets of a graph vertices

Define a function to return all possible subsests for list of vertices V

In [4]:
def subsets(V, prefix=[], arr_sets=[]):
    if V == []: return
    for i in range(len(V)):
        subset = prefix + [V[i]]
        arr_sets += [subset]
        subsets(V[i+1:], subset, arr_sets)

In [5]:
arr_sets = []
subsets(list(G1.keys()), arr_sets=arr_sets)
arr_sets

[[1], [1, 2], [1, 2, 3], [1, 3], [2], [2, 3], [3]]

In [6]:
def check_clique(graph, v_set):
    if len(v_set) < 2: return True
    for i in range(len(v_set)):
        i_set = set(v_set[:i] + v_set[i+1:])
        g_set = graph[v_set[i]]
        if i_set & g_set != i_set :
            return False
    return True

In [7]:
def A_clique_set(graph, v_encode, subsets, A=[]):
    for subset in subsets:
        if check_clique(graph, subset):
            A += [sum([v_encode[v] for v in subset])]

In [8]:
def lp_clique(graph):
    V = list(graph.keys())
    v_encode = {}
    v_num = len(V)
    
    for i in range(v_num):
        temp = np.zeros(v_num)
        temp[i] = -1
        v_encode[V[i]] = temp
    
    arr_sets = []
    subsets(V, arr_sets=arr_sets)
    
    b_clique = np.negative(np.ones(v_num))
    A_clique = []
    A_clique_set(graph, v_encode, arr_sets,A=A_clique)
    A_clique = np.transpose(np.array(A_clique))
    c_clique = np.ones(A_clique.shape[1])
    
    res = linprog(c_clique, A_ub=A_clique, b_ub=b_clique, bounds=(0, None))
    return res, A_clique

In [9]:
lp_clique(G4)

(     con: array([], dtype=float64)
     fun: 1.0
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([0., 0., 0.])
  status: 0
 success: True
       x: array([0., 0., 1., 0., 0., 0., 0.]),
 array([[-1., -1., -1., -1.,  0.,  0.,  0.],
        [ 0., -1., -1.,  0., -1., -1.,  0.],
        [ 0.,  0., -1., -1.,  0., -1., -1.]]))

In [10]:
def subsets_dict(V, prefix=set({}), arr_sets={}):
    if V == []: return 1
    for i in range(len(V)):
        subset = prefix | {V[i]}
        arr_sets[tuple(subset)] = len(arr_sets)
        subsets_dict(V[i+1:], subset, arr_sets)

In [39]:
def A_entropy_set(V, subsets_dict, A=[]):
    num_sets = len(subsets_dict)
    for subset in subsets_dict:
        subset_len = len(subset)
        if subset_len > 1:
            subset_i = subsets_dict[subset]
            a_lq = np.zeros(num_sets)
            a_lq[subset_i] = -1 * (subset_len)
            a_mq = np.zeros(num_sets)
            a_mq[subset_i] = subset_len - 1
            s = set(subset)
            for v in subset:
                sub_i = subsets_dict[tuple(x for x in subset if v != x)]
                a_lq[sub_i] = 1    
                a_mq[sub_i] = -1
            A += [a_lq, a_mq]

In [ ]:
def 

In [40]:
def lp_entropy(graph):
    V = list(graph.keys())
    v_encode = {}
    v_num = len(V)
    
    for i in range(v_num):
        temp = np.zeros(v_num)
        temp[i] = -1
        v_encode[V[i]] = temp
    
    arr_sets = {}
    subsets_dict(V, arr_sets=arr_sets)
    
    A_entropy = []
    A_entropy_set(V, arr_sets, A=A_entropy)
    A_entropy = np.array(A_entropy)
    b_clique = np.zeros(A_entropy.shape[0])
#     c_clique = np.ones(A_clique.shape[1])
    
#     res = linprog(c_clique, A_ub=A_clique, b_ub=b_clique, bounds=(0, None))
    return arr_sets, A_entropy

In [41]:
lp_entropy(G1)

({(1,): 0, (1, 2): 1, (1, 2, 3): 2, (1, 3): 3, (2,): 4, (2, 3): 5, (3,): 6},
 [array([ 1., -2.,  0.,  0.,  1.,  0.,  0.]),
  array([-1.,  1.,  0.,  0., -1.,  0.,  0.]),
  array([ 1.,  0.,  0., -2.,  0.,  0.,  1.]),
  array([-1.,  0.,  0.,  1.,  0.,  0., -1.]),
  array([ 0.,  1., -3.,  1.,  0.,  1.,  0.]),
  array([ 0., -1.,  2., -1.,  0., -1.,  0.]),
  array([ 0.,  0.,  0.,  0.,  1., -2.,  1.]),
  array([ 0.,  0.,  0.,  0., -1.,  1., -1.])])

In [ ]:
p = {"g":1}

In [ ]:
l = [1,2,3]

In [ ]:
set(tuple(l))

In [ ]:
l

In [ ]:
tuple(x for x in tuple(l) if 3 != x)